In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd

# Importing nltk library for Sentiment score and POS tagging 
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus.reader import TaggedCorpusReader 
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
from nltk.corpus import stopwords

import pickle

# To ignore warnings
import warnings
warnings.filterwarnings("ignore")

import string

import statsmodels.api as sm


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [3]:
%%capture
!pip install mxnet
!pip install gluon
!pip install gluonnlp
!pip install emoji

In [4]:
from mxnet import gluon
from mxnet import nd
import gluonnlp as nlp
import re
import math
import emoji
from nltk.tokenize import sent_tokenize, word_tokenize

In [5]:
glove_6b100d = nlp.embedding.create('glove', source='glove.6B.100d')

Embedding file glove.6B.100d.npz is not found. Downloading from Gluon Repository. This may take some time.


Post Caption Analysis using Deep Learning

In [6]:
vocab = nlp.Vocab(nlp.data.Counter(glove_6b100d.idx_to_token))
vocab.set_embedding(glove_6b100d)

In [8]:

df = pd.read_excel('/content/drive/MyDrive/Mental_health_prediction/Data_preparation/DatasetFinal.xlsx')
df  = df.iloc[108:109, ]
text = str(df['firstComment'])
text

'108    Before you criticize someone, ask yourself, wh...\nName: firstComment, dtype: object'

In [9]:
emj = word_tokenize(text)
emj
emoj = [c for c in emj if c in emoji.UNICODE_EMOJI]
emoj

[]

In [10]:
from keras.preprocessing.text import text_to_word_sequence
from nltk.stem import PorterStemmer
import string

sample = []
entity_pre = ['@','#']
    #tokenize the text, lowering and filtering
for tt in df['firstComment']:
  result = text_to_word_sequence(tt, filters='!"$%&()*+,-./:;<=>?[\\]^_`{|}~\t\n', lower=True)
for j in result:
  if j[0] not in entity_pre:    #removing # and @
    sample.append(j)
vtext = [word for word in sample if word.isalpha()]
print(vtext)

['before', 'you', 'criticize', 'someone', 'ask', 'yourself', 'what', 'have', 'you', 'down', 'in', 'life', 'is', 'such', 'a', 'patriot', 'that', 'he', 'gave', 'up', 'his', 'lavish', 'billionaire', 'lifestyle', 'to', 'take', 'criticism', 'from', 'every', 'direction', 'and', 'constantly', 'being', 'attacked', 'by', 'the', 'people', 'of', 'this', 'nation', 'helping', 'he', 'take', 'one', 'cent', 'he', 'gets', 'paid', 'for', 'the', 'job', 'and', 'lost', 'net', 'worth', 'since', 'being', 'in', 'office', 'the', 'first', 'not', 'one', 'to', 'not', 'gain', 'in', 'net', 'worth', 'from', 'the', 'presidency', 'but', 'to', 'actually', 'loose', 'bless', 'this', 'man', 'and', 'his', 'soul', 'for', 'what', 'doing', 'i', 'truly', 'believe', 'god', 'sent', 'him', 'to', 'protect', 'us']


In [11]:
def cos_sim(x, y):
    return nd.dot(x, y) / (nd.norm(x) * nd.norm(y))


def dot_prod(x, y):
  return nd.dot(x, y)



def softmax(x):
    c = 0
    for i in x:
      c = c + np.exp(i)
    d = []
    for i in x:
      d.append(np.exp(i) / c)
    return d



def sig(x):
  return 1/(1+np.exp(-x))



def lg(x):
  return math.log(x)

In [12]:
def prob(rel, word):
  tx = []
  for i in rel:
    if (dot_prod(vocab.embedding[i], vocab.embedding[word]) == 0.0):
      continue
    si = cos_sim(vocab.embedding[i], vocab.embedding[word])
    tx.append(si.asnumpy()[0])
    #print(softmax(tx))
  ab = softmax(tx)
  ab = [round(x,3) for x in ab]
  return ab

In [13]:

with (open('/content/drive/MyDrive/Mental_health_prediction/Model_train/angry_weights.pickle','rb')) as ang:
  U_A = pickle.load(ang)
  W_A = pickle.load(ang)
  b_A = pickle.load(ang)
  H1_A = pickle.load(ang)
  H2_A = pickle.load(ang)
print(U_A,W_A,b_A,H1_A,H2_A,sep='\n')
print('\n')
with (open('/content/drive/MyDrive/Mental_health_prediction/Model_train/happy_weights.pickle','rb')) as hap:
  U_H = pickle.load(hap)
  W_H = pickle.load(hap)
  b_H = pickle.load(hap)
  H1_H = pickle.load(hap)
  H2_H = pickle.load(hap)
print(U_H,W_H,b_H,H1_H,H2_H,sep='\n')
print('\n')
with (open('/content/drive/MyDrive/Mental_health_prediction/Model_train/depression_weights.pickle','rb')) as dep:
  U_D = pickle.load(dep)
  W_D = pickle.load(dep)
  b_D = pickle.load(dep)
  H1_D = pickle.load(dep)
  H2_D = pickle.load(dep)
print(U_D,W_D,b_D,H1_D,H2_D,sep='\n')
print('\n')
with (open('/content/drive/MyDrive/Mental_health_prediction/Model_train/fear_weights.pickle','rb')) as fea:
  U_F = pickle.load(fea)
  W_F = pickle.load(fea)
  b_F = pickle.load(fea)
  H1_F = pickle.load(fea)
  H2_F = pickle.load(fea)
print(U_F,W_F,b_F,H1_F,H2_F,sep='\n')

[0.5735305088552471, 0.5735305088552471, 0.5735305088552471, 0.5735305088552471, 0.5735305088552471, 0.5735305088552471]
[0.5735305088552471, 0.5735305088552471, 0.5735305088552471, 0.5735305088552471, 0.5735305088552471, 0.5735305088552471]
[0.14494800251411338, 0.14494800251411338, 0.14494800251411338, 0.14494800251411338, 0.14494800251411338, 0.14494800251411338]
0.24515719320893398
0.2532973109535338


[0.5008046020020307, 0.5008046020020307, 0.5008046020020307, 0.5008046020020307, 0.5008046020020307, 0.5008046020020307]
[0.5004783679213356, 0.5004783679213356, 0.5004783679213356, 0.5004783679213356, 0.5004783679213356, 0.5004783679213356]
[0.1397904097390755, 0.1397904097390755, 0.1397904097390755, 0.1397904097390755, 0.1397904097390755, 0.1397904097390755]
0.21032951746028322
0.19081880123924788


[0.46623864182830554, 0.46623864182830554, 0.46623864182830554, 0.46623864182830554, 0.46623864182830554, 0.46623864182830554]
[0.47961405033516247, 0.47961405033516247, 0.4796140503351

In [14]:
def ht_value(h1, h2, text, word, U, W, b):
  s_t1_prev, s_t2_prev = h1, h2
  prod = []
  alpha = []
  simil = []
  gru_in = []
  H1, H2 = [], []
  for i in text:
    if (dot_prod(vocab.embedding[i], vocab.embedding[word]) == 0.0):
      continue
    prd = dot_prod(vocab.embedding[i], vocab.embedding[word])
    sil = cos_sim(vocab.embedding[i], vocab.embedding[word])
    prod.append(prd.asnumpy()[0])
    simil.append(sil.asnumpy()[0])
  #print(prod,'\n\n')
  #print(simil,'\n\n')
  alpha = softmax(prod)
  #print(alpha,'\n\n')
  gru_in = np.multiply(alpha, simil)
  #gru_in = np.squeeze(g)
  #print(gru_in)
  for i in range(len(gru_in)):
    z_t1 = 1/(1+np.exp(-1*(U[0]*(gru_in[i]) + W[0]*(s_t1_prev) + b[0])))
    r_t1 = 1/(1+np.exp(-1*(U[1]*(gru_in[i]) + W[1]*(s_t1_prev) + b[1])))
    c_t1 = np.tanh(U[2]*(gru_in[i]) + W[2]*(s_t1_prev * r_t1) + b[2])
    s_t1 = (1 - z_t1) * c_t1 + z_t1 * s_t1_prev
    s_t1_prev = s_t1
    H1.append(s_t1)
  for i in range(len(gru_in)-1,-1,-1):
    z_t2 = 1/(1+np.exp(-1*(U[3]*(gru_in[i]) + W[3]*(s_t2_prev) + b[0])))
    r_t2 = 1/(1+np.exp(-1*(U[4]*(gru_in[i]) + W[4]*(s_t2_prev) + b[1])))
    c_t2 = np.tanh(U[5]*(gru_in[i]) + W[5]*(s_t2_prev * r_t1) + b[2])
    s_t2 = (1 - z_t1) * c_t2 + z_t2 * s_t2_prev
    s_t2_prev = s_t2
    H2.append(s_t2)
  #print('\n',z_t1,r_t1,c_t1,s_t1,sep='     ')
  #print('\n',z_t2,r_t2,c_t2,s_t2,sep='     ')
  #print('\n\n')
  #print(H1,H2,sep='\n\n')
  return H1, H2

In [15]:
#text = ['i','am','extremely','depressed','😞']
#text = ['i','am','extremely','depressed','and','no','one','belongs','to','me']
#text = ['i','am','extremely','happy','and','feeling','proud','of','my','friend','keep','up','the','good','work','🤗']
#text = ['happy', 'happy','happy','happy','happy','happy','happy','happy','happy','happy','happy']
#text = ['i','am','too','scared','to','step','out','in','this','world']
H_E = ['😀','😃','😄','😁','😆','🤣','😂','😇','🥰','🤪','🤗','😎']
A_E = ['😒','🙄','😡','🤬','😠','😈']
D_E = ['😑','😔','😕','😟','☹','🥺','😞']
F_E = ['😨','😰','😱','☠']
flag = 0
h1_A, h2_A = [], []
h1_H, h2_H = [], []
h1_D, h2_D = [], []
h1_F, h2_F = [], []
h1_A, h2_A = ht_value(H1_A, H2_A, text, 'angry', U_A, W_A, b_D)
h1_H, h2_H = ht_value(H1_H, H2_H, text, 'happy', U_H, W_H, b_H)
h1_D, h2_D = ht_value(H1_D, H2_D, text, 'depression', U_D, W_D, b_D)
h1_F, h2_F = ht_value(H1_D, H2_D, text, 'fear', U_F, W_F, b_F)
i = len(h1_A) - 1
print(h1_A[i], h1_H[i], h1_D[i], h1_F[i], sep='   ')
print(h2_A[i], h2_H[i], h2_D[i], h2_F[i], sep='   ')
print(max(h1_A[i], h1_D[i], h1_H[i], h1_F[i]))
print(max(h2_A[i], h2_H[i], h2_D[i], h2_F[i]))
if max(h1_A[i], h1_D[i], h1_H[i], h1_F[i]) == h1_A[i] or max(h2_A[i], h2_H[i], h2_D[i], h2_F[i]) == h2_A[i]:
  print("\nLabel is angry ",sig(max(h1_A[i], h2_A[i])))
  if max(h1_D[i], h1_H[i], h1_F[i]) == h1_H[i] or max(h2_H[i], h2_D[i], h2_F[i]) == h2_H[i]:
    print("\nLabel is happy too ",sig(max(h1_H[i], h2_H[i])))
  elif max(h1_D[i], h1_H[i], h1_F[i]) == h1_D[i] or max(h2_H[i], h2_D[i], h2_F[i]) == h2_D[i]:
    print("\nLabel is depressed too ",sig(max(h1_D[i], h2_D[i])))
  elif max(h1_D[i], h1_H[i], h1_F[i]) == h1_F[i] or max(h2_H[i], h2_D[i], h2_F[i]) == h2_F[i]:
    print("\nLabel is fear too ",sig(max(h1_F[i], h2_F[i])))
  for fg in text:
    if fg in A_E:
      flag = 1
  #if emoj in A_E:
    #flag = 1
elif max(h1_A[i], h1_D[i], h1_H[i], h1_F[i]) == h1_H[i] or max(h2_A[i], h2_H[i], h2_D[i], h2_F[i]) == h2_H[i]:
  print("\nLabel is happy ",sig(max(h1_H[i], h2_H[i])))
  if max(h1_A[i], h1_D[i], h1_F[i]) == h1_A[i] or max(h2_A[i], h2_D[i], h2_F[i]) == h2_A[i]:
    print("\nLabel is angry too ",sig(max(h1_A[i], h2_A[i])))
  elif max(h1_A[i], h1_D[i], h1_F[i]) == h1_D[i] or max(h2_A[i], h2_D[i], h2_F[i]) == h2_D[i]:
    print("\nLabel is depressed too ",sig(max(h1_D[i], h2_D[i])))
  elif max(h1_A[i], h1_D[i], h1_F[i]) == h1_F[i] or max(h2_A[i], h2_D[i], h2_F[i]) == h2_F[i]:
    print("\nLabel is fear too ",sig(max(h1_F[i], h2_F[i])))
  for fg in text:
    if fg in H_E:
      flag = 1
  #if emoj in H_E:
    #flag = 1
elif max(h1_A[i], h1_D[i], h1_H[i], h1_F[i]) == h1_D[i] or max(h2_A[i], h2_H[i], h2_D[i], h2_F[i]) == h2_D[i]:
  print("\nLabel is depressed ",sig(max(h1_D[i], h2_D[i])))
  if max(h1_A[i], h1_H[i], h1_F[i]) == h1_A[i] or max(h2_A[i], h2_H[i], h2_F[i]) == h2_A[i]:
    print("\nLabel is angry too ",sig(max(h1_A[i], h2_A[i])))
  elif max(h1_A[i], h1_H[i], h1_F[i]) == h1_H[i] or max(h2_A[i], h2_H[i], h2_F[i]) == h2_H[i]:
    print("\nLabel is happy too ",sig(max(h1_H[i], h2_H[i])))
  elif max(h1_A[i], h1_H[i], h1_F[i]) == h1_F[i] or max(h2_A[i], h2_H[i], h2_F[i]) == h2_F[i]:
    print("\nLabel is fear too ",sig(max(h1_F[i], h2_F[i])))
  for fg in text:
    if fg in D_E:
      flag = 1
  #if emoj in D_E:
    #flag = 1
elif max(h1_A[i], h1_D[i], h1_H[i], h1_F[i]) == h1_F[i] or max(h2_A[i], h2_H[i], h2_D[i], h2_F[i]) == h2_F[i]:
  print("\nLabel is fear ",sig(max(h1_F[i], h2_F[i])))
  if max(h1_D[i], h1_H[i], h1_A[i]) == h1_H[i] or max(h2_H[i], h2_D[i], h2_A[i]) == h2_H[i]:
    print("\nLabel is happy too ",sig(max(h1_H[i], h2_H[i])))
  elif max(h1_D[i], h1_H[i], h1_A[i]) == h1_D[i] or max(h2_H[i], h2_D[i], h2_A[i]) == h2_D[i]:
    print("\nLabel is depressed too ",sig(max(h1_D[i], h2_D[i])))
  elif max(h1_D[i], h1_H[i], h1_A[i]) == h1_A[i] or max(h2_H[i], h2_D[i], h2_A[i]) == h2_A[i]:
    print("\nLabel is angry too ",sig(max(h1_A[i], h2_A[i])))
  for fg in text:
    if fg in F_E: 
      flag = 1
  #if emoj in F_E:
    #flag = 1
else:
  print('default')
print('\nFlag : ',flag)

0.2560170806823905   0.19075534427306656   0.23713823823883845   0.22965406411021067
0.257234710580448   0.19158955646838904   0.23758922381369757   0.2304057998002622
0.2560170806823905
0.257234710580448

Label is angry  0.5639564013240949

Label is depressed too  0.5591194661246621

Flag :  0


In [16]:
#text = ['i','am','extremely','happy','and','feeling','proud','of','my','friend','keep','up','the','good','work']
print('TEXT',end='\t\t\t')
for i in vtext:
  print(i,end='\t\t')
print('\n')
dem = []
dem = prob(vtext, 'angry')
#print(dem)
print('Angry ',end='\t\t\t')
for i in dem:
  print(i,end='\t\t')
print('\n')
dem.clear()
dem = prob(vtext, 'happy')
print('Happy ',end='\t\t\t')
for i in dem:
  print(i,end='\t\t')
print('\n')
dem.clear()
dem = prob(vtext, 'depression')
print('Depression ',end='\t\t')
for i in dem:
  print(i,end='\t\t')
print('\n')
dem.clear()
dem = prob(vtext, 'fear')
print('Fear ',end='\t\t\t')
for i in dem:
  print(i,end='\t\t')
print('\n')

TEXT			before		you		criticize		someone		ask		yourself		what		have		you		down		in		life		is		such		a		patriot		that		he		gave		up		his		lavish		billionaire		lifestyle		to		take		criticism		from		every		direction		and		constantly		being		attacked		by		the		people		of		this		nation		helping		he		take		one		cent		he		gets		paid		for		the		job		and		lost		net		worth		since		being		in		office		the		first		not		one		to		not		gain		in		net		worth		from		the		presidency		but		to		actually		loose		bless		this		man		and		his		soul		for		what		doing		i		truly		believe		god		sent		him		to		protect		us		

Angry 			0.011		0.011		0.011		0.011		0.012		0.01		0.012		0.011		0.011		0.011		0.01		0.01		0.01		0.01		0.011		0.009		0.011		0.011		0.011		0.011		0.011		0.009		0.009		0.008		0.01		0.011		0.013		0.011		0.011		0.009		0.011		0.011		0.012		0.013		0.012		0.011		0.013		0.01		0.011		0.01		0.01		0.011		0.011		0.011		0.008		0.011		0.011		0.01		0.01		0.011		0.01		0.011		0.011		0.008		0.009		0.01		0.012		0.01		